In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv

In [2]:
# Initial imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [3]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [4]:
path = Path('../Resources/ftd_key_list_sorted.pkl')
ftd_key_list = load_obj(path)
path = Path('../Resources/etf_key_list_sorted.pkl')
etf_key_list = load_obj(path)
path = Path('../Resources/eqt_key_list_sorted.pkl')
eqt_key_list = load_obj(path)

In [5]:
print('All list length:',len(ftd_key_list))
print('ETF list length:',len(etf_key_list))
print('EQT list length:',len(eqt_key_list))

All list length: 3695
ETF list length: 1872
EQT list length: 1823


In [34]:
test_symbol = ftd_key_list[1]
symbol = test_symbol
symbol

'AAAU'

In [35]:
## Load Data
# df = pd.read_csv(
#     '../FilesExportIndividualStockDFs_Big/'+symbol+'_combined_df.csv')
# df
path = Path('../FilesExport_Finished/'+symbol+'_finished_df.pkl')
data_import = load_obj(path)
df = data_import['dataFrame'].copy()
df

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,changeOverTime,QUANTITY_FAILS,ShortVolumeNSDQ,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE
Date,,,,,,,,,,,,,,,,,,
2018-08-15,11.840000,11.840000,11.740000,11.740,11.740,27300.0,27300.0,-0.100,-0.845,11.77333,-0.00845,0.0,210.0,0.0,210.0,4816.0,0.0,4816.0
2018-08-16,11.780000,11.800000,11.740000,11.740,11.740,428400.0,428400.0,-0.040,-0.340,11.76000,-0.00340,0.0,2159.0,0.0,2159.0,2641.0,0.0,2646.0
2018-08-17,11.800000,11.820000,11.770000,11.820,11.820,52400.0,52400.0,0.020,0.169,11.80333,0.00169,5327.0,557.0,0.0,752.0,2695.0,0.0,5080.0
2018-08-20,11.880000,11.910000,11.850000,11.900,11.900,28700.0,28700.0,0.020,0.168,11.88667,0.00168,409049.0,4068.0,0.0,4268.0,5638.0,0.0,6643.0
2018-08-21,11.920000,11.950000,11.890000,11.930,11.930,30600.0,30600.0,0.010,0.084,11.92333,0.00084,13618.0,1852.0,0.0,19552.0,695.0,0.0,696.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-25,17.950000,18.000000,17.930000,17.970,17.970,283012.0,283012.0,0.020,0.111,17.96667,0.00111,36579.0,51686.0,0.0,74484.0,7904.0,0.0,15694.0
2021-10-26,17.860001,17.860001,17.719999,17.830,17.830,311600.0,311600.0,-0.030,-0.168,17.80333,-0.00168,2409.0,105390.0,0.0,120331.0,13602.0,0.0,23865.0
2021-10-27,17.830000,17.890000,17.750000,17.885,17.885,290224.0,290224.0,0.055,0.308,17.84167,0.00308,107600.0,5496.0,0.0,18799.0,5751.0,0.0,8000.0


In [36]:
## Set X and y data 
X = df.drop(columns={'close','adjClose'}).values
y = df['close'].values

scaler = StandardScaler().fit(X)
X = scaler.transform(X) 

In [37]:
# load json and create model
file_path = Path('../Model_Data/'+symbol+'_model_2Layers.json')
with open(file_path, "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

# load weights into new model
file_path = Path('../Model_Data/'+symbol+'_model_2Layers_weights.h5')
loaded_model.load_weights(file_path)

df2 = df.copy()
df2['predicted'] = loaded_model.predict(X)
df_pred = df2[['close','predicted']]
df_pred

,close,predicted
Date,,
2018-08-15,11.740,11.708690
2018-08-16,11.740,11.723052
2018-08-17,11.820,11.808107
2018-08-20,11.900,11.894579
2018-08-21,11.930,11.920912
...,...,...
2021-10-25,17.970,18.019924
2021-10-26,17.830,17.827440
2021-10-27,17.885,17.882219
